### FID SCORE

In [ ]:
!pip install pytorch-fid

In [ ]:
from PIL import Image
import os

def resize_images_in_folder(folder_path, target_size=(299, 299)):
    """Resizes all images in a folder to the target size."""
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
            with Image.open(img_path) as img:
                # Resize and convert to RGB if necessary
                img = img.resize(target_size, Image.LANCZOS).convert("RGB")
                img.save(img_path)
                print(f"Resized and saved {img_path}")
        except Exception as e:
            print(f"Error resizing image {img_path}: {e}")

# Ensure both real and generated images are resized to 299x299
resize_images_in_folder('/home/ishita.agarwal/Documents/aitextual/falcon_input')
resize_images_in_folder('/home/ishita.agarwal/Documents/aitextual/output_falcon')

In [ ]:
from pytorch_fid import fid_score

# Path to real images and generated images
path_real = '/home/ishita.agarwal/Documents/aitextual/falcon_input'
path_generated = '/home/ishita.agarwal/Documents/aitextual/output_falcon'

# Calculate FID score
fid_value = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=50, device='cuda', dims=2048)
print(f'FID Score: {fid_value}')

### CLIP for Precision and Recall with CLIP

In [ ]:
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch

# Load the CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Directory containing the generated images
generated_images_dir = "/home/ishita.agarwal/Documents/aitextual/output_falcon"  # Replace with your generated images directory

# Load all generated images from the directory
generated_image_paths = [os.path.join(generated_images_dir, filename) for filename in os.listdir(generated_images_dir) if filename.endswith('.png')]
generated_images = [Image.open(path) for path in generated_image_paths]

# Define the prompt for evaluation
prompt = "a rendering in the style of <falcon_bird>"

def calculate_clip_similarity(images, prompt):
    # Process text prompt
    text_inputs = clip_processor(text=[prompt], return_tensors="pt", padding=True).to(device)
    text_features = clip_model.get_text_features(**text_inputs)

    # Calculate similarity scores
    similarity_scores = []
    for image in images:
        # Process each image and get features
        image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
        image_features = clip_model.get_image_features(**image_inputs)

        # Calculate cosine similarity
        similarity = torch.cosine_similarity(text_features, image_features).item()
        similarity_scores.append(similarity)
    
    # Average similarity score
    avg_similarity = sum(similarity_scores) / len(similarity_scores)
    print(f"Average CLIP Similarity Score: {avg_similarity}")
    return avg_similarity

# Calculate CLIP similarity score for generated images
calculate_clip_similarity(generated_images, prompt)


### ResNET for concept accuracy

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import os
import json

# Load the ResNet model pre-trained on ImageNet
model = models.resnet50(pretrained=True)
model.eval()

# Load ImageNet class labels - assuming you have a JSON file mapping indices to class names
# Download from https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json
with open("imagenet_class_index.json") as f:
    imagenet_classes = {int(key): value[1] for key, value in json.load(f).items()}

# Define transformations for the image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the target concept classes you want to check
concept_classes = ["style"]

# Directory containing the generated images
generated_images_dir = "/home/ishita.agarwal/Documents/aitextual/logo_output"  # Replace with your path

# Load generated image paths
generated_image_paths = [os.path.join(generated_images_dir, filename) 
                         for filename in os.listdir(generated_images_dir) if filename.endswith(('.png', '.jpg', '.jpeg'))]

# Initialize a counter for matching concepts
concept_count = 0

# Process each generated image
for image_path in generated_image_paths:
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)  # Preprocess and add batch dimension

    # Forward pass through ResNet model
    with torch.no_grad():
        outputs = model(image)
        _, predicted_idx = outputs.max(1)  # Get the predicted index
        predicted_label = imagenet_classes[predicted_idx.item()]  # Map index to class label

    # Check if the predicted label matches any of the target concepts
    if any(concept in predicted_label for concept in concept_classes):
        concept_count += 1

# Calculate concept accuracy score
concept_accuracy = concept_count / len(generated_image_paths) if generated_image_paths else 0
print(f"Concept Accuracy Score: {concept_accuracy}")
